In [25]:
# radimo klasifikaciju na skupu baloons.csv naivnim bajesom. Treba jos da ispisemo ukupnu preciznost i
# izvestaj o klasifikaciji (za svaku klasu preciznost, odziv i f1) za test skup

import pandas as pd

from sklearn.model_selection import train_test_split
# ima vise verzija naivnog bajesa, a ovaj kategoricki je bas ono sto smo radili na papiru
# ovaj je pogodan kada su nam i atributi kategoricki
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [26]:
df = pd.read_csv('../baloons.csv')
df.describe()

,color,size,act,age,inflated
count,76,76,76,76,76
unique,2,2,2,2,2
top,YELLOW,SMALL,DIP,CHILD,F
freq,40,40,38,38,41


In [27]:
df.head()

,color,size,act,age,inflated
0,YELLOW,SMALL,STRETCH,ADULT,T
1,YELLOW,SMALL,STRETCH,ADULT,T
2,YELLOW,SMALL,STRETCH,CHILD,F
3,YELLOW,SMALL,DIP,ADULT,F
4,YELLOW,SMALL,DIP,CHILD,F


In [28]:
# primetimo da columns varaca Index objekat, zato moramo od njega da pravimo listu 
feature_names = df.columns[:-1].tolist()  
X = df[feature_names]
y = df.iloc[:,-1]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(53, 4)
(23, 4)
(53,)
(23,)


In [30]:
# pri izgradnji modela mozemo da prosledimo one alfa i d parametre za Laplase smoothing ako su potrebni
model = CategoricalNB()

In [31]:
# CategoricalNB je dobar kada su nam atributi kategoricki, kao sto nama jesu, ali on trazi da oni
# bas budu neki brojevi, tako da pre fitovanja moramo da svakoj kategoriji dodelimo neku integer oznaku
# za to mozemo da korsitimo OrdinalEncoder (slicno kao za Scaler, to nije model tako da nema predict)
oe = OrdinalEncoder()
oe.fit(X_train)

# mozemo da vidimo koji broj je kategorije su pronadjene za koji atribut
print(oe.categories_)

# kao za Scaler, sada radimo izmenu naseg trin skupa 
X_train = oe.transform(X_train)
# obvezno izmenimo i test skup na isti nacin (vidi u proslom primeru zasto)
X_test = oe.transform(X_test)

[array(['PURPLE', 'YELLOW'], dtype=object), array(['LARGE', 'SMALL'], dtype=object), array(['DIP', 'STRETCH'], dtype=object), array(['ADULT', 'CHILD'], dtype=object)]


In [32]:
model.fit(X_train, y_train)

CategoricalNB()

In [33]:
y_predicted = model.predict(X_test)
print(accuracy_score(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))

0.6086956521739131
[[8 4]
 [5 6]]
              precision    recall  f1-score   support

           F       0.62      0.67      0.64        12
           T       0.60      0.55      0.57        11

    accuracy                           0.61        23
   macro avg       0.61      0.61      0.61        23
weighted avg       0.61      0.61      0.61        23



In [34]:
# mozemo da vidimo one brojeve koje smo racunali rucno na papiru, svaki niz koji se dobije je za 
# jedan atribut, a svaki niz u tom nizu je broj instanci u nekoj kategoriji,
# npr niz koji se dobije ovde za prvi atribut je [[16, 13], [8, 16]]
# prvi 16 je P(color=PURPLE | true), 13 je P(color=YELLOW | true)
# 8 je P(color=PURPLE | false), 16 je P(color=YELLOW | false)
# i tako slicno za svaki
print(model.category_count_)

[array([[18., 11.],
       [ 7., 17.]]), array([[13., 16.],
       [ 9., 15.]]), array([[20.,  9.],
       [ 5., 19.]]), array([[ 8., 21.],
       [18.,  6.]])]


In [35]:
# vidimo i koliko puta je true a koliko puta je false
print(model.class_count_)

[29. 24.]


In [37]:
# mi smo ovde kao i uvek do sada imali neko preprocesiranje, pravljenje modela, fit, predict itd
# imamo jos jedan nacin da to uradimo, a to je preko Pipeline, kome mi mozemo odmah na pocetku da kazemo
# operacije koje hocemo da radimo, npr skaliranje, ecodiranje, pa bajesa itd itd i onda jednom pozovemo 
# fit i predict i on uradi sve sto treba
# pipeline je zgodan jer na nekim mestima moze da eliminise neku nasu ljudsku gresku, npr da mi negde slucajno
# pri treniranju koristimo podatke iz test skupa itd
from sklearn.pipeline import Pipeline

# argumenti su lista transformacija koje se rade(transformacijje su nesto sto ima fit i transform)
# i na kraju liste jedan estimator (to je nesto sto ima fit i predict), tj jedan klasifikator
# listu koraka mozemo da damo i kao obicnu listu npr [OrdinalEncoder(), Scaler()], a mozemo i kao 
# listu tuple-ova gde im damo neka imena
pipe = Pipeline( [('encoder', OrdinalEncoder()), 
                  ('classifier', CategoricalNB)])

# NOTE: ovde opet pozivamo train test split da bi dobili trening i test skupove na kojima nisu izvrsene 
# one transformacije koje smo radili iznad, tj da bi ovaj primer imao smisla
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, stratify=y)

# kada pozovemo ovaj fit, bice pozvani fit i transform za sve transformacije
# i na kraju fit za estimator
# NOTE: ovaj poziv mi daje gresku, iako je sve isto kao na vezbama, ne vidim zasto
pipe.fit(X_train, y_train)

# kada smo uradili ovaj fit, mozemo pristupiti svakom koraku(mozemo i preko imena), i onda iz njega vaditi 
# one parametre isto kao pre, npr:
print(pipe['encoder'].categories_)
print(pipe['classifier'].category_count_)
print(pipe['classifier'].class_count_)

# kada su izvrsene sve transformacije i fitovan model, pozivamo sami predict
y_predicted = pipe.predict(X_test)
print(accuracy_score(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))

TypeError: fit() missing 1 required positional argument: 'y'